In [131]:
import pandas as pd
import numpy as np
import datetime
import re
import csv
from datetime import datetime

def findDelimiter(csvFile):
    '''Find delimiter
          
      Args:
          csvFile: csv file path
      Returns:
          delimiter
    '''
    try:
        csv_file = open(csvFile,encoding="utf8")
    except IOError as e:
        print(f'Unable to open the file {csvFile}')
        print(e)
    
    dialect = csv.Sniffer().sniff(csv_file.read())
    csv_file.seek(0)
    csv_file.close()
    return dialect.delimiter


In [132]:
csvFileName = "25142.csv"

In [133]:
df= pd.read_csv(csvFileName,encoding="utf8", sep=findDelimiter(csvFileName))
df

,Comunidad autónoma de residencia,Tipo de dato,"Gastos totales, medios, distribución e índice",Periodo,Total
0,Total Nacional,Dato base,Gasto total,2020,"0000000508.738.158,54"
1,Total Nacional,Dato base,Gasto total,2019,"566.814.579,02"
2,Total Nacional,Dato base,Gasto total,2018,"556.432.591,36"
3,Total Nacional,Dato base,Gasto total,2017,"540.347.372,65"
4,Total Nacional,Dato base,Gasto total,2016,"520.119.184,84"
...,...,...,...,...,...
5395,19 Melilla,Variación respecto al año base,Índice sobre la media del gmp,2010,NaN
5396,19 Melilla,Variación respecto al año base,Índice sobre la media del gmp,2009,NaN
5397,19 Melilla,Variación respecto al año base,Índice sobre la media del gmp,2008,NaN
5398,19 Melilla,Variación respecto al año base,Índice sobre la media del gmp,2007,NaN


## Identificar NaN

In [134]:
df.isnull().values.any()
df.isnull().sum()

Comunidad autónoma de residencia                    0
Tipo de dato                                        0
Gastos totales, medios, distribución e índice       0
Periodo                                             0
Total                                            1360
dtype: int64

In [135]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()
print(df[df.isnull().any(axis=1)][null_columns].head())

    Total
104   NaN
105   NaN
106   NaN
107   NaN
108   NaN


In [136]:
df.iloc[104]

Comunidad autónoma de residencia                                     Total Nacional
Tipo de dato                                     Variación respecto al año anterior
Gastos totales, medios, distribución e índice                           Gasto total
Periodo                                                                        2006
Total                                                                           NaN
Name: 104, dtype: object

## Identificar Columnas Duplicadas

In [137]:
duplicates = []

In [138]:

    for columns in range(df.shape[1]):
        contents = df.iloc[:, columns]
        for comp in range(columns + 1, df.shape[1]):
            if contents.equals(df.iloc[:, comp]):
                duplicates.append(comp)
                    
duplicates = np.unique(duplicates).tolist()
duplicates

[]

In [139]:
df.dtypes

Comunidad autónoma de residencia                 object
Tipo de dato                                     object
Gastos totales, medios, distribución e índice    object
Periodo                                           int64
Total                                            object
dtype: object

## Identificar Fecha

In [140]:
for row in df:
    for i in row:
         # check if cell is date YYYY-MM-DD or YYYY-MM-DD:HH:MM:SS
        if i.count('-') == 2:
            try:
                datetime.strptime(i, '%Y-%m-%d')
            except ValueError:
                    print(i, "is not a valid date")
                    if(i.count(':')>=2):
                        try:
                            datetime.strptime(i, '%Y-%m-%d:%H:%M:%S')
                        except ValueError:
                            print(i, "is not a valid date")


In [141]:
for row in df:
    for col in row:
        if ' ' in col:
            print(row)

Comunidad autónoma de residencia
Comunidad autónoma de residencia
Comunidad autónoma de residencia
Tipo de dato
Tipo de dato
Gastos totales, medios, distribución e índice
Gastos totales, medios, distribución e índice
Gastos totales, medios, distribución e índice
Gastos totales, medios, distribución e índice
Gastos totales, medios, distribución e índice


In [149]:
def checkNumber(value):
    # Se puede convertir 009 --> 9
    try:
        int(value)
    except Exception:
        return False
    # Se pueden pasar numeros, en tal caso esta formateado    
    if(type(value) == int): 
        return False
    # Es String y es numerico
    return (not not (re.match(r"[0][0-9]*",value)) ) or value == "0"



def checkFechas(value):
    if not (type(value) == str):
        return False
    # Puede ser fecha al tener - y /
    if value.count('-') == 2 or value.count('/') == 2 :
        # ¿Puede se hora con :?
        if ":" in value:
            try:
                datetime.strptime(value, "%Y-%m-%d:%H:%M:%S")
            except ValueError:
                return True
        else:
            try:
                datetime.strptime(value, "%Y-%m-%d")
            except ValueError:
                return True
    return False
checkFechas("2020-12-07:05:02:06")

#datetime.strptime("2020-12-07", "%Y-%m-%d")      
checkNumber("0")


True

In [147]:
dictErrs = {
    "fechas" : [],
    "numbers": [],
}
for i in df.columns:
    dictErrs["numbers"] += [x for x in df[i] if checkNumber(x)]
    dictErrs["fechas"] += [x for x in df[i] if checkFechas(x)]
print(dictErrs)

{'fechas': [], 'numbers': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']}
